# SPARK SQL

## DataFrames

### Creating Dataframes from Python Pandas

In [55]:
import pandas as pd

In [62]:
pandas_df = pd.read_csv("/resources/data/iris.csv")

In [63]:
pandas_df.sample()

,sepal_length,sepal_width,petal_length,petal_width,species
23,5.1,3.3,1.7,0.5,setosa


In [65]:
sqlContext

In [66]:
spark_df = sqlContext.createDataFrame(pandas_df)

In [68]:
spark_df.show(3)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 3 rows



### DataFrame from CSV files

#### 1. Using Row

In [82]:
# Load a text file and convert each line to a tuple.
from pyspark.sql import Row
    
data = sc.textFile('/resources/data/iris.csv')
header = data.first() #extract header
fields = (data.filter(lambda line: line != header) \
              .map(lambda line: line.split(',')) \
              .map(lambda line : Row(sepal_length=float(line[0]), sepal_width=float(line[1])
                                     , petal_length = float(line[2]), petal_width = float(line[3])
                                     , species = line[4])
                   )
          )                          
# Create DataFrame from RDD. using createDataFrame() or toDF()
df0 = sqlContext.createDataFrame(fields)
df0.show(n=3)

+------------+-----------+------------+-----------+-------+
|petal_length|petal_width|sepal_length|sepal_width|species|
+------------+-----------+------------+-----------+-------+
|         1.4|        0.2|         5.1|        3.5| setosa|
|         1.4|        0.2|         4.9|        3.0| setosa|
|         1.3|        0.2|         4.7|        3.2| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 3 rows




#### 2. Using Schema  - StructType

In [95]:
# Load a text file and convert each line to a tuple.
from pyspark.sql.types import *
    
data = sc.textFile('/resources/data/iris.csv')
header = data.first() #extract header
fields = (
          data.filter(lambda line: line != header) \
              .map(lambda line: line.split(',')) \
              .map(lambda line : [float(line[0]), float(line[1]) \
                                , float(line[2]), float(line[3])\
                                , line[4]]
                  )
          )

schema = StructType(
                [StructField('sepal_length', FloatType(), True),
                 StructField('sepal_width', FloatType(), True),
                 StructField('petal_length', FloatType(), True),
                 StructField('petal_width', FloatType(), True),
                 StructField('species', StringType(), True)])

df0 = sqlContext.createDataFrame(fields,schema)
df0.show(n=3)

+------------+-----------+------------+-----------+-------+
|sepal_length|sepal_width|petal_length|petal_width|species|
+------------+-----------+------------+-----------+-------+
|         5.1|        3.5|         1.4|        0.2| setosa|
|         4.9|        3.0|         1.4|        0.2| setosa|
|         4.7|        3.2|         1.3|        0.2| setosa|
+------------+-----------+------------+-----------+-------+
only showing top 3 rows



#### 3. Using *spark.csv* library

In [100]:
df = (sqlContext.read
                .format('com.databricks.spark.csv')
                .options(header='true', inferschema='true')
                .load('/resources/data/car_crashes.csv')
     )

### Using the datframe

In [107]:
df.printSchema()

root
 |-- total: double (nullable = true)
 |-- speeding: double (nullable = true)
 |-- alcohol: double (nullable = true)
 |-- not_distracted: double (nullable = true)
 |-- no_previous: double (nullable = true)
 |-- ins_premium: double (nullable = true)
 |-- ins_losses: double (nullable = true)
 |-- abbrev: string (nullable = true)



In [108]:
df.show()

+-----+------------------+------------------+------------------+------------------+-----------+----------+------+
|total|          speeding|           alcohol|    not_distracted|       no_previous|ins_premium|ins_losses|abbrev|
+-----+------------------+------------------+------------------+------------------+-----------+----------+------+
| 18.8| 7.332000000000001|              5.64|18.048000000000002|             15.04|     784.55|    145.08|    AL|
| 18.1|             7.421|             4.525|16.290000000000003|            17.014|    1053.48|    133.93|    AK|
| 18.6|              6.51| 5.208000000000001|            15.624|            17.856|     899.47|    110.35|    AZ|
| 22.4|             4.032|             5.824|21.055999999999997|             21.28|     827.34|    142.39|    AR|
| 12.0|               4.2|              3.36|             10.92|             10.68|     878.41|    165.63|    CA|
| 13.6|             5.032|3.8080000000000003|10.743999999999998|             12.92|     

In [110]:
df.select('alcohol').show()

+------------------+
|           alcohol|
+------------------+
|              5.64|
|             4.525|
| 5.208000000000001|
|             5.824|
|              3.36|
|3.8080000000000003|
|             3.888|
|              4.86|
|1.5930000000000002|
| 5.190999999999999|
|               3.9|
|             7.175|
|             4.437|
|             4.352|
|             4.205|
|             3.925|
|             4.272|
|             4.922|
|             6.765|
|              4.53|
+------------------+
only showing top 20 rows



### To run SQL queries, we need to register the dataframe as a table

In [111]:
df.registerTempTable('car_crashes')